<a href="https://colab.research.google.com/github/soan12345/OCR/blob/main/MP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pytesseract pdf2image opencv-python pillow

In [3]:
!apt-get install -y poppler-utils tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  poppler-utils tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 4 newly installed, 0 to remove and 22 not upgraded.
Need to get 5,002 kB of archives.
After this operation, 16.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.6 [186 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 5,002 kB in 2s (2,810 kB/s)
Selecting previously unselected package popp

In [4]:
import pytesseract
from pdf2image import convert_from_path
import re
import logging
from typing import Dict, Any, List, Optional

In [9]:

class BillTableExtractor:
    def __init__(self):
        self.logger = logging.getLogger(__name__)
        logging.basicConfig(level=logging.INFO)


        self.field_patterns = {
            'Consumer_ID[IVRS]': r'IVRS[:\s]*([A-Z0-9]+)',
            'Meter_No_KWH': r'Meter\s*Serial\s*No[:\s]*([A-Z0-9]+)',
            'Old_Account_No': r'Old\s*Service\s*Number[:\s]*([A-Z0-9-]+)',
            #'Service Number':r'Service Number\s*([A-Z0-9-]+)',
            'Knumber':r'Location\s* Code\s*\s*([^\n]+)(?=\s*(?:Old Service Number|$))',
            'Phase': r'Phase\s*Given[:\s]*([A-Z]+)',
            'Sanctioned_Load': r'Load\s*Sanctioned[:\s]*([0-9.]+\s*KW)',
            'Meter_Status': r'Reading\s*Type[:\s]*([A-Z_]+)',
            'Tariff': r'Tariff\s*Class[:\s]*([^\n]+)',
            "mdi": r"Maximum Demand\s*([\d.]+)",
            'Board_Name': r'(Madhya\s*Pradesh\s*Madhya\s*Kshetra\s*Vidyut\s*Vitran\s*Company\s*Ltd\.)',
            'Division': r'Division\s*Name[:\s]*([^\n]+?)(?=\s*(?:Tariff Class|$))',
            'Name_On_Bill': r'Mr\.\s*/\s*Ms\.\s*([^\n]+?)(?=\s*Bill\s*Demand|$)',
            #'address' : r'Address\s+((?:.*?(?:\n|$))*?(?=Total Bill Amount On Due Date|Bill Payment last Date|Employee Number|Mobile Number|\n\s*\n|$))',
            'Bill_No': r'Bill\s*Number[:\s]*([A-Z0-9]+)',
            'Bill_Issue_Date': r'Bill\s*Date[:\s]*(\d{2}-[A-Za-z]+-\d{4})',
            'Current_Reading_Date_KWH': r'Current\s*Read\s*Date\s*(\d{2}-[A-Za-z]+-\d{4})',
            'Meter_Consumption_Unit': r'Units consumed[:\s]*(\d{1,3}(?:,\d{3})*)',
            'Fixed_Charges': r'Fixed\s*Charge\s*(\d+\.\d+)',
            'Energy_Charges': r'Energy\s*Charges\s*(\d+\.\d+)',
            'FPPAS_Charges': r'Epp AS Charges\s*(-?\d+\.\d+)',
            'Current_Bill_Amount': r'Current\s*Month\s*Bill\s*(\d+\.\d+)',
            'Amount_Before_Due_Date': r'Total\s*Amount\s*Payable\s*On\s*Due\s*Date[:\s]*(\d+\.\d+)',
            'Amount_After_Due_Date': r'Total\s*Amount\s*Payable\s*After\s*Due\s*Date[:\s]*(\d+\.\d+)',
            #'Last_Amount_Paid': r'Amount\s*Paid[:\s]*(\d+)',
            'Security_Deposit_Paid': r'Security\s*Amount\s*Deposited[:\s]*(\d+\.\d+)',
            'Govt_Subsidy': r'M\.P\.Govt\.Subsidy Amount\s+([\d\.-]+)',
            'Interest_On_Security_Deposit': r'Interest On Security Deposit \(-\)\s+([\d\.-]+)',
            'Other_Rebates': r'Other Rebates\s*\(-\)\s+([\d\.-]+)',
            'Electricity_Duty': r'Electricity Duty\s+([\d\.-]+)',
            'Metering_Charges': r'Metering Charges\s+([\d\.-]+)',
            'ASD_Installment': r'ASD Instalment\s+([\d\.-]+)',
            'Welding_PF': r'Welding/ PF Surcharge/Incentive\s+([\d\.-]+)',
            'Penal_Charges': r'Penal Charges\s+([\d\.-]+)',
            'TOD_Rebate': r'Other / TOD Rebate / Surcharge\s+([\d\.-]+)',
            'Employee_Rebate': r'Employee Rebate \(-\)\s+([\d\.-]+)',
            #'Load_Factor': r'Lock Credit / Load Factor Rebate \(-\) \s+([\d\.-]+)',
            'Delayed_Payment': r'Previous Month Delayed payment Surcharge\s+([\d\.-]+)',
            'Current_Payable': r'Current Payable Amount\s+([\d\.-]+)',
            'Old_Dues': r'Old Dues / Arrear\s+([\d\.-]+)',
            'Late_Payment': r'Due Date Late Payment Surcharge on \(A\)\s+([\d\.-]+)'

        }


        self.table_columns = [
            'Current_Reading',
            'Previous_Reading',
            'Multiplying_Factor',
            'Power_Factor',
            'Meter_Consumption',
            'Sub_Load_Adj_Units',
            'Total_Units',
            'GMC_Units',
            'Billed_Units'
        ]

    def extract_table_from_text(self, text: str) -> Dict[str, Any]:
        """
        Extract meter reading table data using multiple approaches
        to handle any numbers in the standard table format
        """
        # Method 1: Identify table pattern by structure (most robust)
        # Looking for lines with the right pattern of numbers
        lines = text.split('\n')

        # Storing candidate table rows
        table_candidates = []

        for line in lines:
            # Look for lines with multiple decimal numbers (likely table rows)
            decimal_values = re.findall(r'\d+\.\d+', line)
            integer_values = re.findall(r'(?<!\d)\d+(?!\d\.)', line)  # Find standalone integers

            # Count total numeric values
            total_numeric = len(decimal_values) + len(integer_values)

            # Checking if this line matches our expected table pattern
            # We expect at least 7+ numeric values in a table row
            if total_numeric >= 7:
                self.logger.info(f"Potential table row found: {line}")

                # Extracting all numeric values (both decimal and integer)
                all_values = re.findall(r'\d+\.\d+|\d+', line)

                # If we have enough values to possibly form our table row
                if len(all_values) >= len(self.table_columns):
                    table_candidates.append(all_values)


        if table_candidates:
            table_candidates.sort(key=len, reverse=True)

            best_candidate = table_candidates[0]


            if len(best_candidate) >= 9:
                try:
                    #  a dictionary mapping column names to values
                    table_data = {}

                    # Mapping the first 9 values to our column names
                    for i, column in enumerate(self.table_columns):
                        if i < len(best_candidate):
                            value = best_candidate[i]

                            if column == 'Multiplying_Factor':
                                table_data[column] = int(float(value))
                            else:
                                table_data[column] = float(value)

                    return table_data
                except (ValueError, IndexError) as e:
                    self.logger.warning(f"Error parsing table row values: {e}")



    def extract_from_pdf(self, pdf_path: str, debug: bool = True) -> Dict[str, Any]:

        try:
            # Convert PDF to images
            images = convert_from_path(pdf_path)


            results = {field: None for field in self.field_patterns.keys()}

            for column in self.table_columns:
                results[column] = None


            results['raw_text'] = []

            # Processing each page
            for page_num, image in enumerate(images, 1):
                # Performing OCR
                text = pytesseract.image_to_string(image)

                # Storing the raw text in results
                results['raw_text'].append(f"--- PAGE {page_num} ---\n{text}")

                # if debug:
                #     # Save extracted text for debugging
                #     with open(f'extracted_text_page_{page_num}.txt', 'w', encoding='utf-8') as f:
                #         f.write(text)
                #     self.logger.info(f"Saved extracted text from page {page_num}")

                # Extract fields using patterns
                for field, pattern in self.field_patterns.items():
                    if results[field] is None:
                        match = re.search(pattern, text)
                        if match:
                            results[field] = match.group(1).strip()
                            if debug:
                                self.logger.info(f"Found {field}: {results[field]}")

                # Extract table data
                table_data = self.extract_table_from_text(text)
                if table_data:
                    results.update(table_data)
                    if debug:
                        self.logger.info(f"Extracted table data: {table_data}")

            # Clean up string results (except raw_text)
            results = {k: (v.replace('|', '').strip() if isinstance(v, str) and v and k != 'raw_text' else v)
                      for k, v in results.items()}

            # Joining all raw text pages into a single string
            results['raw_text'] = '\n\n'.join(results['raw_text'])


            if debug:
                missing_fields = [field for field, value in results.items()
                                if value is None and field != 'raw_text']
                if missing_fields:
                    self.logger.warning(f"Failed to extract: {', '.join(missing_fields)}")

            return results

        except Exception as e:
            self.logger.error(f"Error processing PDF: {str(e)}")
            return {"error": str(e), "raw_text": "Failed to extract text due to error"}

def main():
    extractor = BillTableExtractor()

    results = extractor.extract_from_pdf("/content/549_N2022007081_2025-01-18.pdf", debug=True)

    total_fields = len (extractor.field_patterns) + len(extractor.table_columns)
    # displaying results
    print("\nExtracted Fields:")
    print(f"Total Possible Fields: {total_fields}")
    print("-" * 50)
    for field, value in results.items():
        if value is not None:
            print(f"{field}: {value}")
    print("-" * 50)



if __name__ == "__main__":
    main()


Extracted Fields:
Total Possible Fields: 46
--------------------------------------------------
Consumer_ID[IVRS]: N2022007081
Meter_No_KWH: GENUS9207791
Old_Account_No: 39-1-414739382012
Knumber: 2414739 - [ URBAN ]
Phase: THREE
Sanctioned_Load: 20.0 KW
Meter_Status: NORMAL
Tariff: LV2 [LV2.2]
mdi: 6.432
Board_Name: Madhya Pradesh Madhya Kshetra Vidyut Vitran Company Ltd.
Division: Narmadapuram
Name_On_Bill: SMT SUNITA
Bill_No: JAN25N002416987
Bill_Issue_Date: 18-Jan-2025
Current_Reading_Date_KWH: 01-Jan-2025
Meter_Consumption_Unit: 848
Fixed_Charges: 5328.00
Energy_Charges: 5855.06
FPPAS_Charges: -264.48
Current_Bill_Amount: 12083.64
Amount_Before_Due_Date: 11859.00
Amount_After_Due_Date: 12007.00
Security_Deposit_Paid: 29438.00
Govt_Subsidy: 0.00
Interest_On_Security_Deposit: 162.87
Other_Rebates: 62.17
Electricity_Duty: 819.00
Metering_Charges: 0.00
ASD_Installment: 0.00
Welding_PF: 559.05
TOD_Rebate: -213.00
Employee_Rebate: 0.00
Delayed_Payment: 0.00
Current_Payable: 11859.00
Old